In [ ]:
import pandas as pd
for stock_df in (fb, amzn, aapl, ibm):
	stock_df['Normed Return'] = stock_df['Adj. Close'] /stock_df.iloc[0]['Adj. Close']
for stock_df, allo in zip((fb, amzn, aapl, ibm),[.2,.4,.3,.1]):
	stock_df['Allocation'] = stock_df['Normed Return']*allo
# value of each position
for stock_df in (fb, amzn, aapl, ibm):
	stock_df['Position Value'] = stock_df['Allocation']*1000000
# create list of all position values
all_pos_vals = [fb['Position Value'], amzn['Position Value'], aapl['Position Value'], ibm['Position Value']]

# concatenate the list of position values
portfolio_val = pd.concat(all_pos_vals, axis=1)

# set the column names
portfolio_val.columns = ['FB', 'AMZN', 'AAPL', 'IBM']

# add a total portfolio column
portfolio_val['Total'] = portfolio_val.sum(axis=1)

# plot our portfolio
import matplotlib.pyplot as plt
%matplotlib inline
portfolio_val['Total'].plot(figsize=(10,8))
portfolio_val.drop('Total',axis=1).plot(figsize=(10,8))

# Daily Return
portfolio_val['Daily Return'] = portfolio_val['Total'].pct_change(1)

# average daily return
portfolio_val['Daily Return'].mean()

# standard deviation
portfolio_val['Daily Return'].std()

# plot histogram of daily returns
portfolio_val['Daily Return'].plot(kind='hist', bins=50, figsize=(4,5))

# cumulative portfolio return
cum_return = 100 * (portfolio_val['Total'][-1]/portfolio_val['Total'][0] - 1)

sharpe_ratio = portfolio_val['Daily Return'].mean() / portfolio_val['Daily Return'].std()

ASR = (252**0.5) * sharpe_ratio

# get just the Adj Close column of FB, AMZN, AAPL, IBM
fb = q.get('WIKI/FB.11', start_date=start, end_date=end)
amzn = q.get('WIKI/AMZN.11', start_date=start, end_date=end)
aapl = q.get('WIKI/AAPL.11', start_date=start, end_date=end)
ibm = q.get('WIKI/IBM.11', start_date=start, end_date=end)

# concatenate them and rename the columns
stocks = pd.concat([fb,amzn,aapl,ibm], axis=1)
stocks.columns = ['fb','amzn','aapl','ibm']

# arithmetic mean daily return
stocks.pct_change(1).mean()

# arithmetic daily return
stocks.pct_change(1).head()

# log daily return
log_return = np.log(stocks/stocks.shift(1))

print(stocks.columns)

weights = np.array(np.random.random(4))
print('Random Weights:')
print(weights)

print('Rebalance')
weights = weights/np.sum(weights)print(weights)

# expected return
print('Expected Portfolio Return')
exp_ret = np.sum((log_return.mean()*weights)*252)
print(exp_ret)

# expected volatility
print('Expected Volatility')
exp_vol = np.sqrt(np.dot(weights.T,np.dot(log_return.cov()*252, weights)))
print(exp_vol)

# Sharpe Ratio
print('Sharpe Ratio')
SR = exp_ret/exp_vol
print(SR)

num_ports = 5000
all_weights = np.zeros((num_ports, len(stocks.columns)))
ret_arr = np.zeros(num_ports)
vol_arr = np.zeros(num_ports)
sharpe_arr = np.zeros(num_ports)

for ind in range(num_ports):
    # weights
    weights = np.array(np.random.random(4))
    weights = weights/np.sum(weights)

    # save the weights
    all_weights[ind,:] = weights

    # expected return
    ret_arr[ind] = np.sum((log_return.mean()*weights)*252)

    # expected volatility
    vol_arr[ind] = np.sqrt(np.dot(weights.T,np.dot(log_return.cov()*252, weights)))

    # Sharpe Ratio
    sharpe_arr[ind] = ret_arr[ind]/vol_arr[ind]

# plot the data
plt.figure(figsize=(12,8))
plt.scatter(vol_arr,ret_arr,c=sharpe_arr,cmap='plasma')
plt.colorbar(label='Sharpe Ratio')
plt.xlabel('Volatility')
plt.ylabel('Return')

max_sr_ret = ret_arr[4988]
max_sr_vol = vol_arr[4988]

# plot the dataplt.figure(figsize=(12,8))
plt.scatter(vol_arr,ret_arr,c=sharpe_arr,cmap='plasma')
plt.colorbar(label='Sharpe Ratio')plt.xlabel('Volatility')
plt.ylabel('Return')

# add a red dot for max_sr_vol & max_sr_ret
plt.scatter(max_sr_vol, max_sr_ret, c='red', s=50, edgecolors='black')

def get_ret_vol_sr(weights):
	weights = np.array(weights)
    ret = np.sum(log_ret.mean() * weights) * 252
    vol = np.sqrt(np.dot(weights.T,np.dot(log_ret.cov()*252,weights)))
    sr = ret/vol return np.array([ret,vol,sr])

from scipy.optimize import minimize

# minimize negative Sharpe Ratio
def neg_sharpe(weights):
	return get_ret_vol_sr(weights)[2] * -1

# check allocation sums to 1
def check_sum(weights):
	return np.sum(weights) - 1

# create constraint variable
cons = ({'type':'eq','fun':check_sum})

# create weight boundaries
bounds = ((0,1),(0,1),(0,1),(0,1))

# initial guess
init_guess = [0.25, 0.25, 0.25, 0.25]

opt_results = minimize(neg_sharpe, init_guess, method='SLSQP', bounds=bounds, constraints=cons)
